In [ ]:
import ROOT, os, sys, glob
from math import *
ROOT.EnableImplicitMT(4);

In [ ]:
# List of files we have
list(glob.glob("/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/*125X_v0.0.root"))

In [ ]:
# Load one data file into RDataframe  https://root.cern.ch/doc/master/classROOT_1_1RDataFrame.html
rdf = ROOT.RDataFrame("Events", "/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/l1MuNano_HAHM_ZdToMuMu_M15_PU200.125X_v0.0.root")
rdf.Describe()

In [ ]:
## Now we make a plot of the number of generated muons and the ones reconstructed by L1T
hNGen = rdf.Histo1D(("nGenMu","N(#mu)_{gen}",7,-0.5,6.5), "GenMu_mass")
hNL1 = rdf.Histo1D(("nL1Mu","N(#mu)_{L1}",7,-0.5,6.5), "TkMu_mass")
entries = rdf.Count().GetValue()
print(f"Processed {entries} entries.")
## And plot it
c1 = ROOT.TCanvas("c1","c1", 1200, 400)
c1.Divide(2,1)
c1.cd(1)
hNGen.Draw()
c1.cd(2)
hNL1.Draw()
c1.Draw()

In [ ]:
## Now we make pairs of muons
## This needs a bit of inlined C++ code
def makeDimuonPairs(name, rdf):
    ## define 4-vectors in cartesian coordinates, for easier 
    rdf = rdf.Define(f"{name}Mu_p4", f"ROOT::VecOps::Construct<ROOT::Math::XYZTVector>(ROOT::VecOps::Construct<ROOT::Math::PtEtaPhiMVector>({name}Mu_pt,{name}Mu_eta,{name}Mu_phi,{name}Mu_mass))")
    ## make indices of all the pairs
    rdf = rdf.Define(f"{name}DiMu_indices", f"""
        std::vector<std::pair<int,int>> ret; 
        for (int i = 0, n = n{name}Mu; i < n; ++i) 
            for (int i2 = i+1; i2 < n; ++i2) 
                ret.emplace_back(i,i2);
        return ret;""")
    ## Make the number of dimuons, for convenience later
    rdf = rdf.Define(f"n{name}DiMu", f"{name}DiMu_indices.size()")
    ## compute the mass, charge
    rdf = rdf.Define(f"{name}DiMu_mass", f"""
        ROOT::RVecF ret; 
        for (const auto & pair : {name}DiMu_indices) 
            ret.emplace_back(({name}Mu_p4[pair.first]+{name}Mu_p4[pair.second]).M());
        return ret;""")
    rdf = rdf.Define(f"{name}DiMu_charge", f"""
        ROOT::RVecI ret; 
        for (const auto & pair : {name}DiMu_indices) 
            ret.emplace_back({name}Mu_charge[pair.first]+{name}Mu_charge[pair.second]);
        return ret;""")
    return rdf;


rdf_gendimu = makeDimuonPairs("Gen", rdf)

## Now we make a plot of the number of generated muons and the ones reconstructed by L1T
hNGenPairs = rdf_gendimu.Histo1D(("nGenDiMu","N(#mu#mu)_{gen};N(#mu#mu)_{gen};Events",7,-0.5,6.5), "nGenDiMu")
hGenPairMass = rdf_gendimu.Histo1D(("GenMass","m(#mu#mu)_{gen};m (GeV);Pairs",100,0.0,20.0), "GenDiMu_mass")
hGenPairCharge = rdf_gendimu.Histo1D(("GenCharge","q(#mu#mu)_{gen};q(#mu_{1}) + q(#mu_{2});Pairs",7,-3.5,3.5), "GenDiMu_charge")
entries = rdf_gendimu.Count().GetValue()
print(f"Processed {entries} entries.")
## And plot it
c1 = ROOT.TCanvas("c1","c1", 1500, 400)
c1.Divide(3,1)
for i, h in enumerate([hNGenPairs,hGenPairMass,hGenPairCharge]):
    c1.cd(i+1)
    h.Draw()
c1.Draw()

In [ ]:
## Repeat the same for L1 reconstructed pairs
rdf_l1dimu = makeDimuonPairs("Tk", rdf_gendimu)

## Now we make a plot of the number of generated muons and the ones reconstructed by L1T
hNL1Pairs = rdf_l1dimu.Histo1D(("nTkDiMu","N(#mu#mu)_{L1};N(#mu#mu)_{L1};Events",7,-0.5,6.5), "nTkDiMu")
hL1PairMass = rdf_l1dimu.Histo1D(("TkMass","m(#mu#mu)_{L1};m (GeV);Pairs",100,0.0,20.0), "TkDiMu_mass")
hL1PairCharge = rdf_l1dimu.Histo1D(("TkCharge","q(#mu#mu)_{L1};q(#mu_{1}) + q(#mu_{2});Pairs",7,-3.5,3.5), "TkDiMu_charge")
entries = rdf_l1dimu.Count().GetValue()
print(f"Processed {entries} entries.")
## And plot it
c1 = ROOT.TCanvas("c1","c1", 1500, 400)
c1.Divide(3,1)
for i, h in enumerate([hNL1Pairs,hL1PairMass,hL1PairCharge]):
    c1.cd(i+1)
    h.Draw()
c1.Draw()

In [ ]:
## Now we add a few more variables

def makeMoreDimuonVariables(name, rdf):
    ## compute the separation along the beam line
    rdf = rdf.Define(f"{name}DiMu_dz", f"""
        ROOT::RVecF ret; 
        for (const auto & pair : {name}DiMu_indices) 
            ret.emplace_back(({name}Mu_vz[pair.first]-{name}Mu_vz[pair.second]));
        return ret;""")
    rdf = rdf.Define(f"{name}DiMu_minPt", f"""
        ROOT::RVecF ret; 
        for (const auto & pair : {name}DiMu_indices) 
            ret.emplace_back(std::min({name}Mu_pt[pair.first],{name}Mu_pt[pair.second]));
        return ret;""")
    return rdf;

rdf_dimu = makeMoreDimuonVariables("Tk", makeMoreDimuonVariables("Gen", rdf_l1dimu))
## Now we make a plot of the number of generated muons and the ones reconstructed by L1T
hGenPairDZ = rdf_dimu.Histo1D(("GenDZ","dz(#mu#mu)_{L1};z(#mu_{1}) - z(#mu_{2}) (cm);Pairs",100,-10,10), "GenDiMu_dz")
hL1PairDZ = rdf_dimu.Histo1D(("TkDZ","dz(#mu#mu)_{L1};z(#mu_{1}) - z(#mu_{2}) (cm);Pairs",100,-10,10), "TkDiMu_dz")
hGenPairMinPt = rdf_dimu.Histo1D(("GenMinPt","min p_{T} (Gen);min(p_{T}(#mu_{1}), p_{T}(#mu_{2})) (GeV);Pairs",100,0,10), "GenDiMu_minPt")
hL1PairMinPt = rdf_dimu.Histo1D(("TkMinPt","min p_{T} (L1);min(p_{T}(#mu_{1}), p_{T}(#mu_{2})) (GeV);Pairs",100,0,10), "TkDiMu_minPt")
entries = rdf_dimu.Count().GetValue()
print(f"Processed {entries} entries.")
## And plot it
c1 = ROOT.TCanvas("c1","c1", 1200, 800)
c1.Divide(2,2)
for i, h in enumerate([hGenPairDZ,hL1PairDZ,hGenPairMinPt,hL1PairMinPt]):
    c1.cd(i+1)
    h.Draw()
c1.Draw()

In [ ]:
# d=ROOT.TFile::ls(Option_t = "/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout")
# d.Describe()